### This file contains an easy testing of torch-mlir tools with python notebook


#### Adding required dependencies

torch-mlir imports

In [8]:
import torch
import torchvision

import torch_mlir
from torch_mlir.dialects.torch.importer.jit_ir import ClassAnnotator, ModuleBuilder
from torch_mlir.dialects.torch.importer.jit_ir.torchscript_annotations import extract_annotations

from torch_mlir.passmanager import PassManager
from torch_mlir_e2e_test.linalg_on_tensors_backends.refbackend import RefBackendLinalgOnTensorsBackend

general dependencies

In [9]:
import requests
from PIL import Image

import numpy as np

Utiilities

In [10]:
def compile_and_load_on_refbackend(module):
    """Compile an MLIR Module to an executable module.

    This uses the Torch-MLIR reference backend which accepts
    linalg-on-tensors as the way to express tensor computations.
    """
    backend = RefBackendLinalgOnTensorsBackend()
    compiled = backend.compile(module)
    return backend.load(compiled)

Basic testing of how torch-mlir handle multiple modules

In [11]:
class TempModule(torch.nn.Module):
    def forward(self, a):
        b = torch.add(a, a)
        c = torch.mul(a, b)
        return c

class StupidModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.a = TempModule()
        self.b = TempModule()
    
    def forward(self, x):
        y = self.a(x)
        z = self.b(y)
        return z

compiled = torch_mlir.compile(TempModule(), torch.ones(3), output_type=torch_mlir.OutputType.LINALG_ON_TENSORS)
print(compiled)

error: unhandled: could not import node:  = prim::Store[name="x"](%x) # /tmp/ipykernel_8855/1947926451.py:13:22



RuntimeError: see diagnostics